In [5]:
import os
import pickle
import numpy as np
import pandas as pd
from skimage import io, measure, color, morphology
from skimage.feature import local_binary_pattern
from skimage.filters import gabor
from scipy import ndimage as ndi
import warnings
import importlib

warnings.filterwarnings("ignore")

# ---------- Auto-detect greycomatrix / greycoprops ----------
def locate_glcm():
    """Try multiple import paths for greycomatrix/greycoprops depending on skimage version.
       Returns tuple (greycomatrix_func, greycoprops_func) or (None, None) if unavailable."""
    candidates = [
        ("skimage.feature.texture", "greycomatrix", "greycoprops"),
        ("skimage.feature", "greycomatrix", "greycoprops"),
        ("skimage.feature._greycomatrix", "greycomatrix", "greycoprops"),
        ("skimage.feature._greycomatrix", "greycomatrix", "greycoprops"),
    ]
    for module_name, fn1, fn2 in candidates:
        try:
            mod = importlib.import_module(module_name)
            gcm = getattr(mod, fn1, None)
            gcp = getattr(mod, fn2, None)
            if gcm is not None and gcp is not None:
                return gcm, gcp
        except Exception:
            continue
    # last resort: try to import from skimage.feature and check attributes
    try:
        import skimage.feature as feat
        gcm = getattr(feat, "greycomatrix", None)
        gcp = getattr(feat, "greycoprops", None)
        if gcm is not None and gcp is not None:
            return gcm, gcp
    except Exception:
        pass
    return None, None

greycomatrix_func, greycoprops_func = locate_glcm()
if greycomatrix_func is None or greycoprops_func is None:
    print("Warning: greycomatrix/greycoprops not found in this skimage version. GLCM features will be skipped.")
else:
    print("Using greycomatrix from:", greycomatrix_func.__module__)

# ----------------- helper utils -----------------
def _safe_uint8(img):
    arr = np.asarray(img)
    if arr.dtype == np.uint8:
        return arr
    mn, mx = float(arr.min()), float(arr.max())
    if mx > mn:
        out = ((arr - mn) / (mx - mn) * 255.0).astype(np.uint8)
    else:
        out = np.zeros_like(arr, dtype=np.uint8)
    return out

def preprocess_img_for_feats(img):
    """Normalize to single-channel uint8, handle RGBA/multipage TIFF etc."""
    arr = np.asarray(img)
    # multipage TIFF returned as (frames, H, W) -> take first frame
    if arr.ndim == 3 and arr.shape[0] > 1 and (arr.shape[2] not in (3,4)):
        arr = arr[0]
    # RGBA -> drop alpha channel
    if arr.ndim == 3 and arr.shape[2] == 4:
        arr = arr[..., :3]
    # RGB -> grayscale
    if arr.ndim == 3 and arr.shape[2] == 3:
        try:
            g = color.rgb2gray(arr)  # returns float 0..1
            arr = (g * 255).astype(np.uint8)
        except Exception:
            arr = arr.mean(axis=2).astype(np.uint8)
    # normalize floats/scalars
    if arr.dtype != np.uint8:
        arr = _safe_uint8(arr)
    return arr

# ---------- feature functions ----------
def compute_glcm_features_safe(img8, distances=(1,), angles=(0,)):
    feats = {}
    if greycomatrix_func is None:
        # fill with NaNs for expected keys
        for prop in ['contrast','correlation','energy','homogeneity','ASM']:
            feats[f'glcm_{prop}'] = np.nan
        return feats
    try:
        im = img8
        if im.ndim == 3:
            im = color.rgb2gray(im)
            im = _safe_uint8(im)
        glcm = greycomatrix_func(im, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
        for prop in ['contrast','correlation','energy','homogeneity','ASM']:
            try:
                val = greycoprops_func(glcm, prop).mean()
            except Exception:
                val = np.nan
            feats[f'glcm_{prop}'] = float(val) if val is not None else np.nan
    except Exception:
        for prop in ['contrast','correlation','energy','homogeneity','ASM']:
            feats[f'glcm_{prop}'] = np.nan
    return feats

def compute_lbp_stats(img8, P=8, R=1):
    try:
        im = img8
        if im.ndim == 3:
            im = color.rgb2gray(im)
        im8 = _safe_uint8(im)
        lbp = local_binary_pattern(im8, P, R, method='uniform')
        hist, _ = np.histogram(lbp, bins=int(lbp.max()+1), density=True)
        return {'lbp_mean': float(hist.mean()), 'lbp_var': float(hist.var()), 'lbp_entropy': float(-np.sum(hist[hist>0]*np.log(hist[hist>0]+1e-9)))}
    except Exception:
        return {'lbp_mean': np.nan, 'lbp_var': np.nan, 'lbp_entropy': np.nan}

def compute_gabor_stats(img8, freqs=(0.1,0.2), thetas=(0,)):
    feats = {}
    try:
        im = img8
        if im.ndim == 3:
            im = color.rgb2gray(im)
        imgf = _safe_uint8(im).astype(float)
        for f in freqs:
            for t in thetas:
                try:
                    real, imag = gabor(imgf, frequency=f, theta=t)
                    mag = np.sqrt(real**2 + imag**2)
                    feats[f'gabor_f{f}_t{int(t*180/np.pi)}_mean'] = float(np.nanmean(mag))
                    feats[f'gabor_f{f}_t{int(t*180/np.pi)}_std'] = float(np.nanstd(mag))
                except Exception:
                    feats[f'gabor_f{f}_t{int(t*180/np.pi)}_mean'] = np.nan
                    feats[f'gabor_f{f}_t{int(t*180/np.pi)}_std'] = np.nan
    except Exception:
        pass
    return feats

def radial_ps_features(img8, nbins=20):
    feats = {}
    try:
        im = img8
        if im.ndim == 3:
            im = color.rgb2gray(im)
        imf = _safe_uint8(im).astype(float)
        imf = imf - imf.mean()
        F = np.fft.fftshift(np.fft.fft2(imf))
        psd2d = np.abs(F)**2
        y, x = np.indices(psd2d.shape)
        cx, cy = psd2d.shape[1]//2, psd2d.shape[0]//2
        r = np.hypot(x-cx, y-cy).astype(int)
        rmax = max(1, r.max())
        bins = np.linspace(0, rmax, nbins+1).astype(int)
        rad_mean = []
        for i in range(nbins):
            mask = (r>=bins[i]) & (r<bins[i+1])
            if mask.sum() == 0:
                rad_mean.append(0.0)
            else:
                rad_mean.append(psd2d[mask].mean())
        rad_mean = np.array(rad_mean)
        feats['radial_ps_mean'] = float(rad_mean.mean())
        feats['radial_ps_std'] = float(rad_mean.std())
        feats['radial_ps_q25'] = float(np.percentile(rad_mean, 25))
        feats['radial_ps_q50'] = float(np.percentile(rad_mean, 50))
        feats['radial_ps_q75'] = float(np.percentile(rad_mean, 75))
    except Exception:
        feats.update({k: np.nan for k in ['radial_ps_mean','radial_ps_std','radial_ps_q25','radial_ps_q50','radial_ps_q75']})
    return feats

def skeleton_features(mask):
    feats = {}
    try:
        binm = (mask>0).astype(bool)
        sk = morphology.skeletonize(binm)
        skel_len = int(sk.sum())
        kernel = np.ones((3,3), int)
        neigh = ndi.convolve(sk.astype(int), kernel, mode='constant', cval=0)
        branch_pts = int(((sk==1) & (neigh-1>2)).sum())
        end_pts = int(((sk==1) & (neigh-1==1)).sum())
        feats['skel_len'] = skel_len
        feats['skel_branch_pts'] = branch_pts
        feats['skel_end_pts'] = end_pts
    except Exception:
        feats['skel_len'] = np.nan
        feats['skel_branch_pts'] = np.nan
        feats['skel_end_pts'] = np.nan
    return feats

def object_level_features(img8, mask):
    feats = {}
    try:
        lab = measure.label(mask>0)
        props = ['area','perimeter','eccentricity','solidity','major_axis_length','minor_axis_length','orientation','mean_intensity','max_intensity','min_intensity']
        tab = measure.regionprops_table(lab, intensity_image=img8, properties=props)
        df = pd.DataFrame(tab)
        if df.shape[0] == 0:
            feats['n_objects'] = 0
            feats['area_mean'] = 0
            return feats
        feats['n_objects'] = int(df.shape[0])
        feats['area_mean'] = float(df['area'].mean())
        feats['area_median'] = float(df['area'].median())
        feats['area_std'] = float(df['area'].std())
        feats['area_q25'] = float(df['area'].quantile(0.25))
        feats['area_q75'] = float(df['area'].quantile(0.75))
        feats['eccentricity_mean'] = float(df['eccentricity'].mean())
        feats['solidity_mean'] = float(df['solidity'].mean())
        feats['major_axis_mean'] = float(df['major_axis_length'].mean())
        feats['minor_axis_mean'] = float(df['minor_axis_length'].mean())
        feats['mean_intensity_mean'] = float(df['mean_intensity'].mean())
    except Exception:
        feats['n_objects'] = 0
    return feats

# --------------- main runner -----------------
RESULTS_PKL = "results.pkl"
OUT_IMAGE_CSV = "image_features.csv"
OUT_PEROBJ_CSV = "per_object.csv"

if not os.path.exists(RESULTS_PKL):
    raise FileNotFoundError("results.pkl not found in working dir. Run Cellpose script first.")

with open(RESULTS_PKL, "rb") as f:
    results = pickle.load(f)
print("Loaded", len(results), "records from", RESULTS_PKL)

image_rows = []
perobj_rows = []

for rec in results:
    img_path = rec.get('image_path')
    mask = rec.get('mask')
    if mask is None:
        mp = rec.get('mask_path')
        if mp and os.path.exists(mp):
            try:
                mask = io.imread(mp)
            except Exception:
                try:
                    mask = np.load(mp + ".npy")
                except Exception:
                    print("Cannot load mask for", rec.get('image_name'))
                    continue
        else:
            print("No mask for record", rec.get('run'))
            continue

    try:
        img = io.imread(img_path)
    except Exception:
        print("Cannot read image", img_path)
        continue

    img8 = preprocess_img_for_feats(img)

    base = {
        'run': rec.get('run'),
        'molecule_id': rec.get('molecule_id'),
        'image_name': rec.get('image_name'),
        'image_path': img_path,
        'mask_path': rec.get('mask_path'),
        'dia': rec.get('dia'),
        'flow_threshold': rec.get('flow_threshold'),
        'cellprob_threshold': rec.get('cellprob_threshold'),
        'blur': rec.get('blur')
    }

    # features
    olf = object_level_features(img8, mask)
    skf = skeleton_features(mask)
    glcm = compute_glcm_features_safe(img8)
    lbp = compute_lbp_stats(img8)
    gaborf = compute_gabor_stats(img8)
    radial = radial_ps_features(img8)

    rec_feat = {}
    rec_feat.update(base)
    rec_feat.update(olf)
    rec_feat.update(skf)
    rec_feat.update(glcm)
    rec_feat.update(lbp)
    rec_feat.update(gaborf)
    rec_feat.update(radial)
    image_rows.append(rec_feat)

    # per-object details
    lab = measure.label(mask>0)
    props = measure.regionprops(lab, intensity_image=img8)
    for pid, p in enumerate(props):
        perobj_rows.append({
            'run': rec.get('run'),
            'molecule_id': rec.get('molecule_id'),
            'image_name': rec.get('image_name'),
            'obj_id': pid,
            'area': p.area,
            'perimeter': p.perimeter,
            'eccentricity': p.eccentricity,
            'solidity': p.solidity,
            'mean_intensity': p.mean_intensity
        })

# save
df_img = pd.DataFrame(image_rows)
df_img.to_csv(OUT_IMAGE_CSV, index=False)
print("Saved image-level features to", OUT_IMAGE_CSV)

if perobj_rows:
    pd.DataFrame(perobj_rows).to_csv(OUT_PEROBJ_CSV, index=False)
    print("Saved per-object features to", OUT_PEROBJ_CSV)
else:
    print("No per-object features extracted.")


Loaded 27 records from results.pkl
Saved image-level features to image_features.csv
Saved per-object features to per_object.csv
